In [1]:
%load_ext autoreload
%autoreload 2

In [3]:
import os
import pandas as pd
import numpy as np

## Interpolate Undefined Values by Zip
This notebook groups the data by zipcode and then performs linear interpolation on each column to fill in undefined values.

In [38]:
filename = '../../data/interim/2.0-output.csv'
df = pd.read_csv(filename)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1109656 entries, 0 to 1109655
Data columns (total 11 columns):
 #   Column       Non-Null Count    Dtype  
---  ------       --------------    -----  
 0   year         1109656 non-null  int64  
 1   zipcode      1109656 non-null  int64  
 2   EQI_zip      824710 non-null   float64
 3   SFR_zip      824710 non-null   float64
 4   RECPI_zip    824710 non-null   float64
 5   EQI_MSA      688481 non-null   float64
 6   SFR_MSA      688481 non-null   float64
 7   RECPI_MSA    688481 non-null   float64
 8   EQI_state    944005 non-null   float64
 9   SFR_state    944005 non-null   float64
 10  RECPI_state  944005 non-null   float64
dtypes: float64(9), int64(2)
memory usage: 93.1 MB


In [39]:
df = df.groupby('zipcode').apply(lambda group: group.interpolate())

In [40]:
df = df.groupby('zipcode').apply(lambda group: group.fillna(method='backfill'))

In [41]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1109656 entries, 0 to 1109655
Data columns (total 11 columns):
 #   Column       Non-Null Count    Dtype  
---  ------       --------------    -----  
 0   year         1109656 non-null  int64  
 1   zipcode      1109656 non-null  int64  
 2   EQI_zip      1109656 non-null  float64
 3   SFR_zip      1109656 non-null  float64
 4   RECPI_zip    1109656 non-null  float64
 5   EQI_MSA      692288 non-null   float64
 6   SFR_MSA      692288 non-null   float64
 7   RECPI_MSA    692288 non-null   float64
 8   EQI_state    1109656 non-null  float64
 9   SFR_state    1109656 non-null  float64
 10  RECPI_state  1109656 non-null  float64
dtypes: float64(9), int64(2)
memory usage: 93.1 MB


### Fill missing MSA with mean of zip and state

In [42]:
df.EQI_MSA = df.apply(lambda row: (row.EQI_zip + row.EQI_state)/2 if np.isnan(row.EQI_MSA) else row.EQI_MSA, axis=1)

In [44]:
df.SFR_MSA = df.apply(lambda row: (row.SFR_zip + row.SFR_state)/2 if np.isnan(row.SFR_MSA) else row.SFR_MSA, axis=1)

In [45]:
df.RECPI_MSA = df.apply(lambda row: (row.RECPI_zip + row.RECPI_state)/2 if np.isnan(row.RECPI_MSA) else row.RECPI_MSA, axis=1)

In [46]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1109656 entries, 0 to 1109655
Data columns (total 11 columns):
 #   Column       Non-Null Count    Dtype  
---  ------       --------------    -----  
 0   year         1109656 non-null  int64  
 1   zipcode      1109656 non-null  int64  
 2   EQI_zip      1109656 non-null  float64
 3   SFR_zip      1109656 non-null  float64
 4   RECPI_zip    1109656 non-null  float64
 5   EQI_MSA      1109656 non-null  float64
 6   SFR_MSA      1109656 non-null  float64
 7   RECPI_MSA    1109656 non-null  float64
 8   EQI_state    1109656 non-null  float64
 9   SFR_state    1109656 non-null  float64
 10  RECPI_state  1109656 non-null  float64
dtypes: float64(9), int64(2)
memory usage: 93.1 MB


## Output DataFrame to File

In [47]:
filename = '../../data/processed/na-data-fill.csv'
df.to_csv(filename, index=False)